In [1]:
import statsmodels.formula.api as smf
import fastreg as fr
from fastreg import I, R, C, C0

### Generate Data

In [2]:
models = ['linear', 'poisson', 'logit', 'negbin']
data = fr.dataset(N=1_000_000, K1=10, K2=100, models=models, seed=89320432)
data_wide = fr.dataset(N=1_000_000, K1=10, K2=10_000, models=models, seed=89320433)
data.head()

,x1,x2,yhat0,yhat,id1,id2,y0,y,Eb0,Eb,b0,b,Ep0,Ep,p0,p,nb0,nb
0,-1.429225,1.830295,0.769410,2.149410,H,68,1.791671,1.585510,0.683393,0.895614,1,1,2.158492,8.579792,4,10,1,5
1,0.687153,0.795373,0.783369,1.403369,C,42,0.295552,3.163529,0.686406,0.802718,1,1,2.188835,4.068886,1,2,1,0
2,0.764353,-1.060225,-0.306829,-0.036829,C,7,0.381637,-0.639362,0.423889,0.490794,1,1,0.735777,0.963841,0,1,2,0
3,-0.309887,-0.777701,-0.459587,0.800413,D,96,0.143218,1.139854,0.387084,0.690063,0,1,0.631545,2.226461,0,0,1,3
4,-0.616042,-0.752266,-0.536172,0.623828,H,46,-0.468395,1.462211,0.369078,0.651089,1,1,0.584983,1.866057,0,1,1,7


In [3]:
# for statsmodels runs
data1 = data.copy()
data1['id2'] = data1['id2'].astype(str)

### Normal OLS

In [4]:
%time fr.ols(y=R.y0, x=I+R.x1+R.x2, data=data)

CPU times: user 57 ms, sys: 62.9 ms, total: 120 ms
Wall time: 22.6 ms


y0,coeff,stderr,low95,high95,pvalue
I,0.101859,0.001000,0.099900,0.103818,0.0
x1,0.301647,0.001000,0.299688,0.303606,0.0
x2,0.599408,0.000999,0.597450,0.601366,0.0


In [5]:
%time smf.ols('y0 ~ 1 + x1 + x2', data=data).fit().params

CPU times: user 355 ms, sys: 1.27 s, total: 1.62 s
Wall time: 130 ms


Intercept    0.101859
x1           0.301647
x2           0.599408
dtype: float64

In [6]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

CPU times: user 436 ms, sys: 925 ms, total: 1.36 s
Wall time: 327 ms


y,coeff,stderr,low95,high95,pvalue
I,0.112463,0.010413,0.092054,0.132872,0.0
x1,0.299557,0.001001,0.297595,0.301518,0.0
x2,0.601840,0.001000,0.599880,0.603800,0.0
id1=B,0.104734,0.004476,0.095962,0.113507,0.0
id1=C,0.201965,0.004474,0.193197,0.210734,0.0
...,...,...,...,...,...
id2=95,0.937461,0.014102,0.909821,0.965101,0.0
id2=96,0.956624,0.014109,0.928970,0.984278,0.0
id2=97,0.960951,0.014099,0.933318,0.988584,0.0
id2=98,0.993398,0.014132,0.965699,1.021097,0.0


In [7]:
%time smf.ols('y ~ 1 + x1 + x2 + id1 + id2', data=data1).fit().params

CPU times: user 1min 34s, sys: 21.1 s, total: 1min 55s
Wall time: 11.1 s


Intercept    0.112463
id1[T.B]     0.104734
id1[T.C]     0.201965
id1[T.D]     0.300212
id1[T.E]     0.403094
               ...   
id2[T.97]    0.960951
id2[T.98]    0.993398
id2[T.99]    0.988670
x1           0.299557
x2           0.601840
Length: 111, dtype: float64

### High Dimensional

In [8]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data_wide)

CPU times: user 4.12 s, sys: 2.5 s, total: 6.62 s
Wall time: 4.57 s


y,coeff,stderr,low95,high95,pvalue
I,0.143877,0.103699,-0.059369,0.347123,1.653041e-01
x1,0.301630,0.001004,0.299662,0.303597,0.000000e+00
x2,0.599284,0.001006,0.597313,0.601255,0.000000e+00
id1=B,0.097642,0.004499,0.088824,0.106461,0.000000e+00
id1=C,0.189008,0.004491,0.180207,0.197810,0.000000e+00
...,...,...,...,...,...
id2=9995,1.045182,0.146196,0.758643,1.331722,8.730794e-13
id2=9996,1.029893,0.146587,0.742588,1.317198,2.128075e-12
id2=9997,0.892911,0.142021,0.614554,1.171267,3.233234e-10
id2=9998,0.961372,0.144705,0.677755,1.244989,3.059886e-11


In [9]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=data_wide)

CPU times: user 376 ms, sys: 64.5 ms, total: 440 ms
Wall time: 363 ms


y,coeff,stderr,low95,high95,pvalue
I,0.143877,0.103699,-0.059369,0.347123,1.653041e-01
x1,0.301630,0.001004,0.299662,0.303597,0.000000e+00
x2,0.599284,0.001006,0.597313,0.601255,0.000000e+00
id1=B,0.097642,0.004499,0.088824,0.106461,0.000000e+00
id1=C,0.189008,0.004491,0.180207,0.197810,0.000000e+00
...,...,...,...,...,...
id2=9995,1.045182,0.146196,0.758643,1.331722,8.730794e-13
id2=9996,1.029893,0.146587,0.742588,1.317198,2.128075e-12
id2=9997,0.892911,0.142021,0.614554,1.171267,3.233234e-10
id2=9998,0.961372,0.144705,0.677755,1.244989,3.059886e-11


In [10]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, absorb=C.id2, data=data_wide)

CPU times: user 753 ms, sys: 1.71 s, total: 2.46 s
Wall time: 352 ms


y,coeff,stderr,low95,high95,pvalue
I,0.605524,0.003035,0.599576,0.611471,0.0
x1,0.301630,0.000998,0.299674,0.303585,0.0
x2,0.599284,0.001014,0.597297,0.601270,0.0
id1=B,0.097642,0.004531,0.088762,0.106523,0.0
id1=C,0.189008,0.004545,0.180100,0.197917,0.0
id1=D,0.295160,0.004506,0.286328,0.303991,0.0
id1=E,0.402802,0.004510,0.393963,0.411641,0.0
id1=F,0.495409,0.004493,0.486602,0.504215,0.0
id1=G,0.596910,0.004467,0.588155,0.605665,0.0
id1=H,0.694326,0.004490,0.685526,0.703125,0.0


### Poisson

In [11]:
%time fr.poisson(y=R.p0, x=I+R.x1+R.x2, data=data)

[  0] ℓ=-0.89843, g=0.75802, Δβ=0.12940, Δℓ=inf, μR=0.12812, μC=nan
[  7] ℓ=-0.62214, g=0.00037, Δβ=0.00000, Δℓ=0.00000, μR=0.33358, μC=nan
CPU times: user 1.26 s, sys: 171 ms, total: 1.43 s
Wall time: 1.31 s


p0,coeff,stderr,low95,high95,pvalue
I,0.098297,0.001023,0.096292,0.100301,0.0
x1,0.304303,0.000849,0.302639,0.305967,0.0
x2,0.598136,0.000848,0.596474,0.599797,0.0


In [12]:
%time fr.poisson(y=R.p, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

[  0] ℓ=-0.21677, g=2.64045, Δβ=0.13064, Δℓ=inf, μR=0.13009, μC=0.12815
[ 10] ℓ=2.55435, g=0.00253, Δβ=0.01367, Δℓ=0.00018, μR=0.41596, μC=0.37655
[ 20] ℓ=2.55464, g=0.00037, Δβ=0.00257, Δℓ=0.00001, μR=0.39920, μC=0.40266
[ 30] ℓ=2.55473, g=0.00023, Δβ=0.00188, Δℓ=0.00001, μR=0.39225, μC=0.41319
[ 40] ℓ=2.55480, g=0.00019, Δβ=0.00163, Δℓ=0.00001, μR=0.38646, μC=0.42192
[ 49] ℓ=2.55485, g=0.00018, Δβ=0.00146, Δℓ=0.00001, μR=0.38186, μC=0.42887
CPU times: user 10.5 s, sys: 2.12 s, total: 12.6 s
Wall time: 7.59 s


p,coeff,stderr,low95,high95,pvalue
I,0.242801,0.006694,0.229682,0.255921,0.0
x1,0.300814,0.000508,0.299818,0.301809,0.0
x2,0.601970,0.000506,0.600978,0.602962,0.0
id1=B,0.096640,0.002832,0.091090,0.102191,0.0
id1=C,0.201290,0.002770,0.195860,0.206721,0.0
...,...,...,...,...,...
id2=95,0.809230,0.007565,0.794403,0.824058,0.0
id2=96,0.819840,0.007595,0.804955,0.834726,0.0
id2=97,0.824663,0.007600,0.809766,0.839559,0.0
id2=98,0.847331,0.007559,0.832515,0.862146,0.0


### Logit

In [13]:
%time fr.logit(y=R.b0, x=I+R.x1+R.x2, data=data)

[  0] ℓ=-0.67995, g=0.11919, Δβ=0.12883, Δℓ=inf, μR=0.11412, μC=nan
[  6] ℓ=-0.64336, g=0.00005, Δβ=0.00053, Δℓ=0.00001, μR=0.33446, μC=nan
CPU times: user 925 ms, sys: 123 ms, total: 1.05 s
Wall time: 744 ms


b0,coeff,stderr,low95,high95,pvalue
I,0.098528,0.002104,0.094404,0.102652,0.0
x1,0.303882,0.002146,0.299676,0.308088,0.0
x2,0.600975,0.002269,0.596527,0.605422,0.0


In [14]:
%time fr.logit(y=R.b, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

[  0] ℓ=-0.64757, g=0.16899, Δβ=0.12827, Δℓ=inf, μR=0.12647, μC=0.12104
[ 10] ℓ=-0.54389, g=0.00036, Δβ=0.01504, Δℓ=0.00005, μR=0.43125, μC=0.34491
[ 20] ℓ=-0.54380, g=0.00010, Δβ=0.00370, Δℓ=0.00000, μR=0.41145, μC=0.37922
[ 30] ℓ=-0.54379, g=0.00007, Δβ=0.00168, Δℓ=0.00000, μR=0.40357, μC=0.39201
[ 40] ℓ=-0.54379, g=0.00007, Δβ=0.00110, Δℓ=0.00000, μR=0.39925, μC=0.39879
[ 44] ℓ=-0.54379, g=0.00007, Δβ=0.00100, Δℓ=0.00000, μR=0.39788, μC=0.40090
CPU times: user 9.43 s, sys: 1.18 s, total: 10.6 s
Wall time: 6.47 s


b,coeff,stderr,low95,high95,pvalue
I,0.292555,0.022796,0.247874,0.337235,0.0
x1,0.303312,0.002382,0.298644,0.307980,0.0
x2,0.597782,0.002508,0.592866,0.602697,0.0
id1=B,0.096958,0.009835,0.077682,0.116235,0.0
id1=C,0.206193,0.009910,0.186770,0.225617,0.0
...,...,...,...,...,...
id2=95,0.690461,0.033460,0.624881,0.756040,0.0
id2=96,0.795693,0.034023,0.729009,0.862377,0.0
id2=97,0.779295,0.033837,0.712976,0.845614,0.0
id2=98,0.783570,0.033997,0.716937,0.850203,0.0


### Ultra Wide

In [15]:
N = 5_000_000
df = pd.DataFrame({ 
    'x1': np.random.rand(N), 
    'x2': np.random.rand(N), 
    'id1': np.ceil(10*np.arange(N)/N+1e-7).astype(int),
    'id2': np.random.randint(1, 100_001, size=N)
})
df['y'] = 1 + 2*df['x1'] + 3*df['x2'] + np.log10(df['id1']) + np.log10(df['id2']) + np.random.randn(N)
print(df[['id1', 'id2']].nunique())

id1        10
id2    100000
dtype: int64


In [16]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=df)

CPU times: user 2.06 s, sys: 832 ms, total: 2.89 s
Wall time: 2.04 s


y,coeff,stderr,low95,high95,pvalue
I,0.953622,0.137249,0.684619,1.222626,3.702150e-12
x1,1.998616,0.001564,1.995552,2.001681,0.000000e+00
x2,2.998819,0.001564,2.995754,3.001884,0.000000e+00
id1=2,0.302079,0.002019,0.298122,0.306035,0.000000e+00
id1=3,0.476832,0.002019,0.472876,0.480788,0.000000e+00
...,...,...,...,...,...
id2=99996,5.069992,0.188337,4.700858,5.439126,0.000000e+00
id2=99997,5.172589,0.196973,4.786529,5.558649,0.000000e+00
id2=99998,5.141093,0.202525,4.744150,5.538035,0.000000e+00
id2=99999,5.091094,0.188337,4.721959,5.460228,0.000000e+00
